In [ ]:
#pip install ultralytics opencv-python numpy torch torchvision torchaudio


In [1]:
from ultralytics import YOLO
import os
import shutil
import random
import glob


In [ ]:


# Définition des chemins
base_path = "detection"
image_source = "Disease"
label_source = "Layout"

# Dossiers des classes
classes = ["Sana", "Fito", "Monilia"]

# Création des dossiers de destination
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

# Récupération des fichiers et séparation en train/val (80%/20%)
train_ratio = 0.8

for class_name in classes:
    image_files = sorted(glob.glob(os.path.join(image_source, class_name, "*.jpg")))  # Modifier si autre format
    label_files = sorted(glob.glob(os.path.join(label_source, class_name, "*.txt")))

    paired_files = list(zip(image_files, label_files))
    random.shuffle(paired_files)

    train_size = int(len(paired_files) * train_ratio)
    
    for i, (img, label) in enumerate(paired_files):
        split = "train" if i < train_size else "val"
        shutil.copy(img, os.path.join(base_path, f"images/{split}", os.path.basename(img)))
        shutil.copy(label, os.path.join(base_path, f"labels/{split}", os.path.basename(label)))

# Création du fichier data.yaml
yaml_content = f"""train: {os.path.abspath(base_path)}/images/train
val: {os.path.abspath(base_path)}/images/val
nc: 3
names: ["Sana", "Fito", "Monilia"]
"""

with open(os.path.join(base_path, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ Organisation des fichiers terminée !")


In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

model = YOLO("yolov8s.pt")  # Utilisation d'un modèle pré-entraîné
model.train(data="detection/data.yaml", epochs=50, imgsz=640, batch=8)


In [ ]:
#import shutil
#shutil.copy("runs/detect/train4/weights/best.pt", "detection/model_cacao.pt")


In [ ]:
from ultralytics import YOLO
import cv2

# Charger le modèle entraîné
model0 = YOLO("detection/model_cacao.pt")

# Charger l'image sur laquelle tu veux détecter les fèves
image_path = "detection/test_image.jpg"
results = model0(image_path)  # Détection sur l'image


In [ ]:
import cv2

# Charger l'image
image = cv2.imread(image_path)

# Vérifier si l'image est correctement chargée
if image is None:
    print(f"Erreur : Impossible de charger l'image à partir de {image_path}")
    exit()

# Dessiner les boîtes de détection
for result in results:
    boxes = result.boxes.xyxy  # Coordonnées des bounding boxes [x1, y1, x2, y2]
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)  # Conversion en entier
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dessiner la boîte verte

# Afficher l'image avec OpenCV
cv2.imshow("Détection des fèves", image)
cv2.waitKey(0)  # Attendre une touche pour fermer la fenêtre
cv2.destroyAllWindows()  # Fermer toutes les fenêtres OpenCV


# Entrainement en utilisant yolo pour la dectection

In [2]:
import os
import glob
import shutil
import random
from ultralytics import YOLO

# --------------------------
# 1. Organisation du dataset
# --------------------------
base_path = "detection"
image_source = "Disease"
label_source = "Layout"
classes = ["Fito","Monilia","Sana"]

# Création des dossiers
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

In [3]:
# Répartition train / val
train_ratio = 0.8

for class_name in classes:
    image_files = sorted(glob.glob(os.path.join(image_source, class_name, "*.jpg")))
    label_files = sorted(glob.glob(os.path.join(label_source, class_name, "*.txt")))

    paired_files = list(zip(image_files, label_files))
    random.shuffle(paired_files)

    train_size = int(len(paired_files) * train_ratio)
    
    for i, (img, label) in enumerate(paired_files):
        split = "train" if i < train_size else "val"
        shutil.copy(img, os.path.join(base_path, f"images/{split}", os.path.basename(img)))
        shutil.copy(label, os.path.join(base_path, f"labels/{split}", os.path.basename(label)))

In [4]:

# --------------------------
# 2. Création du fichier data.yaml
# --------------------------
yaml_content = f"""train: {os.path.abspath(base_path)}/images/train
val: {os.path.abspath(base_path)}/images/val
nc: 3
names: ["Fito","Monilia","Sana"]
"""

with open(os.path.join(base_path, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ Organisation des fichiers terminée !")

✅ Organisation des fichiers terminée !


In [26]:

# --------------------------
# 3. Entraînement du modèle YOLOv8
# --------------------------
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

model = YOLO("yolov8n.pt")  # 🔁 change "s" par "n", "m", ou "l" selon ta capacité

model.train(data="detection/data.yaml", epochs=100, imgsz=640, batch=8)

New https://pypi.org/project/ultralytics/8.3.117 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

train: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\train.cache... 249 images, 0 backgrounds, 0 corrupt: 100%|██████████| 249/249 [00:00<?, ?it/s]
val: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]

Plotting labels to runs\detect\train2\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.484      2.825      1.273         28        640: 100%|██████████| 32/32 [01:53<00:00,  3.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:14<00:00,  3.67s/it]

...

                   all         63        336      0.734      0.655      0.736      0.497



100 epochs completed in 3.409 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:13<00:00,  3.40s/it]


                   all         63        336       0.72      0.724      0.756       0.52
                  Fito         33         45      0.833      0.689      0.722      0.522
               Monilia         22         26      0.619      0.731       0.77      0.593
                  Sana         54        265      0.709      0.751      0.775      0.444
Speed: 2.0ms preprocess, 66.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023FD1E5D730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [27]:
# --------------------------
# 4. Évaluation du modèle
# --------------------------

best_model_path = "runs/detect/train2/weights/best.pt"  

model = YOLO(best_model_path)

metrics = model.val(data=os.path.join(base_path, "data.yaml"), imgsz=640, split="val")

Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:13<00:00,  3.36s/it]


                   all         63        336       0.72      0.724      0.756       0.52
                  Fito         33         45      0.833      0.689      0.722      0.522
               Monilia         22         26      0.619      0.731       0.77      0.593
                  Sana         54        265      0.709      0.751      0.775      0.444
Speed: 1.7ms preprocess, 63.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val6


In [28]:


# --------------------------
# 5. Affichage des métriques
# --------------------------
print("\n📊 Évaluation du modèle :")
print(f"mAP@0.5       : {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95  : {metrics.box.map:.4f}")
print(f"Précision moy : {metrics.box.p.mean():.4f}")
print(f"Recall moy    : {metrics.box.r.mean():.4f}")



📊 Évaluation du modèle :
mAP@0.5       : 0.7560
mAP@0.5:0.95  : 0.5196
Précision moy : 0.7203
Recall moy    : 0.7235


Comparaison avec model.val :
mAP@0.5 (val)       : 0.7572
mAP@0.5:0.95 (val)  : 0.5559
Précision moy (val) : 0.7436
Rappel moy (val)    : 0.6950



📊 Évaluation du modèle (n50)train :
mAP@0.5       : 0.7219
mAP@0.5:0.95  : 0.4932
Précision moy : 0.8182
Recall moy    : 0.6101


📊 Évaluation du modèle (s50) train6:
mAP@0.5       : 0.7445
mAP@0.5:0.95  : 0.4557
Précision moy : 0.7808
Recall moy    : 0.6303

In [8]:
import os
import glob
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import precision_score, recall_score, f1_score

# Charger le modèle YOLOv8s
best_model_path = "runs/detect/train6/weights/best.pt"  # Remplacer par le chemin correct pour YOLOv8s
model = YOLO(best_model_path)

# Paramètres
base_path = "detection"
val_images_path = os.path.join(base_path, "images/val")
conf_threshold = 0.7  # Seuil de confiance
iou_threshold = 0.45  # Seuil IoU pour NMS
iou_match_threshold = 0.3  # Seuil IoU pour appariement (assoupli)

# Récupérer les images de validation
val_images = glob.glob(os.path.join(val_images_path, "*.jpg"))

# Listes pour stocker les prédictions et les vérités terrain
all_preds = []
all_gts = []

# Fonction pour lire les annotations (vérités terrain)
def read_yolo_labels(label_path):
    labels = []
    bboxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                labels.append(class_id)
                bboxes.append([x_center, y_center, width, height])
    return labels, bboxes

# Fonction pour calculer l'IoU entre deux boîtes (format YOLO: [x_center, y_center, width, height])
def calculate_iou(box1, box2):
    # Convertir en [x_min, y_min, x_max, y_max]
    x1_min = box1[0] - box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_max = box1[1] + box1[3] / 2
    
    x2_min = box2[0] - box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_max = box2[1] + box2[3] / 2
    
    # Calculer l'intersection
    x_inter_min = max(x1_min, x2_min)
    y_inter_min = max(y1_min, y2_min)
    x_inter_max = min(x1_max, x2_max)
    y_inter_max = min(y1_max, y2_max)
    
    inter_area = max(0, x_inter_max - x_inter_min) * max(0, y_inter_max - y_inter_min)
    
    # Calculer l'union
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = area1 + area2 - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

# Effectuer les prédictions
for img_path in val_images:
    # Prédire avec seuil de confiance et NMS
    results = model.predict(img_path, conf=conf_threshold, iou=iou_threshold, imgsz=640, verbose=False)
    
    # Récupérer les prédictions
    pred_classes = []
    pred_bboxes = []
    for result in results:
        if result.boxes is not None:
            pred_classes.extend(result.boxes.cls.cpu().numpy().astype(int))
            boxes = result.boxes.xywhn.cpu().numpy()  # xywh normalisé
            pred_bboxes.extend(boxes)
    
    # Récupérer les vérités terrain
    label_path = img_path.replace("images/val", "labels/val").replace(".jpg", ".txt")
    gt_classes, gt_bboxes = read_yolo_labels(label_path)
    
    # Débogage : imprimer pour une image (facultatif)
    if len(val_images) <= 5:  # Limiter à 5 images pour tester
        print(f"\nImage: {img_path}")
        print(f"Pred classes: {pred_classes}")
        print(f"GT classes: {gt_classes}")
    
    # Appariement des boîtes basé sur l'IoU
    matched_preds = []
    matched_gts = []
    used_preds = set()
    used_gts = set()
    
    if pred_bboxes and gt_bboxes:
        # Calculer la matrice IoU
        iou_matrix = np.zeros((len(pred_bboxes), len(gt_bboxes)))
        for i, pred_box in enumerate(pred_bboxes):
            for j, gt_box in enumerate(gt_bboxes):
                iou_matrix[i, j] = calculate_iou(pred_box, gt_box)
        
        # Débogage : imprimer la matrice IoU pour une image
        if len(val_images) <= 5:
            print(f"IoU Matrix:\n{iou_matrix}")
        
        # Appariement : associer les boîtes avec l'IoU le plus élevé
        while np.max(iou_matrix) >= iou_match_threshold:
            max_idx = np.unravel_index(np.argmax(iou_matrix), iou_matrix.shape)
            pred_idx, gt_idx = max_idx
            
            if pred_classes[pred_idx] == gt_classes[gt_idx]:  # Classes correspondantes
                matched_preds.append(pred_classes[pred_idx])
                matched_gts.append(gt_classes[gt_idx])
                used_preds.add(pred_idx)
                used_gts.add(gt_idx)
            
            # Supprimer la boîte utilisée
            iou_matrix[pred_idx, :] = -1
            iou_matrix[:, gt_idx] = -1
    
    # Ajouter les prédictions non appariées comme faux positifs
    for i, pred_class in enumerate(pred_classes):
        if i not in used_preds:
            all_preds.append(pred_class)
            all_gts.append(-1)  # Faux positif
    
    # Ajouter les vérités terrain non appariées comme faux négatifs
    for i, gt_class in enumerate(gt_classes):
        if i not in used_gts:
            all_preds.append(-1)  # Faux négatif
            all_gts.append(gt_class)
    
    # Ajouter les paires appariées
    all_preds.extend(matched_preds)
    all_gts.extend(matched_gts)

# Convertir en tableaux numpy
all_preds = np.array(all_preds)
all_gts = np.array(all_gts)

# Débogage : afficher le nombre de TP, FP, FN
tp = np.sum((all_preds == all_gts) & (all_gts != -1))
fp = np.sum((all_preds != -1) & (all_gts == -1))
fn = np.sum((all_preds == -1) & (all_gts != -1))
print(f"\n📊 Débogage :")
print(f"Vrais positifs (TP) : {tp}")
print(f"Faux positifs (FP)  : {fp}")
print(f"Faux négatifs (FN)  : {fn}")

# Calculer les métriques sans filtrer les FP
# Remplacer -1 par une classe fictive (par exemple, 3) pour inclure FP et FN
all_preds_adjusted = np.where(all_preds == -1, 3, all_preds)
all_gts_adjusted = np.where(all_gts == -1, 3, all_gts)

# Calculer les métriques
precision = precision_score(all_gts_adjusted, all_preds_adjusted, average='weighted', zero_division=0)
recall = recall_score(all_gts_adjusted, all_preds_adjusted, average='weighted', zero_division=0)
f1 = f1_score(all_gts_adjusted, all_preds_adjusted, average='weighted', zero_division=0)

# Afficher les métriques
print(f"\n📊 Métriques avec seuil de confiance = {conf_threshold} (predict):")
print(f"Précision (weighted) : {precision:.4f}")
print(f"Rappel (weighted)    : {recall:.4f}")
print(f"F1-Score (weighted)  : {f1:.4f}")

# Évaluation avec model.val au même seuil (YOLOv8s)
print("\n📊 Comparaison avec model.val :")
print(f"mAP@0.5 (val)       : {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95 (val)  : {metrics.box.map:.4f}")
print(f"Précision moy (val) : {metrics.box.p.mean():.4f}")
print(f"Rappel moy (val)    : {metrics.box.r.mean():.4f}")


📊 Débogage :
Vrais positifs (TP) : 130
Faux positifs (FP)  : 6
Faux négatifs (FN)  : 206

📊 Métriques avec seuil de confiance = 0.7 (predict):
Précision (weighted) : 0.9392
Rappel (weighted)    : 0.3801
F1-Score (weighted)  : 0.5405

📊 Comparaison avec model.val :
mAP@0.5 (val)       : 0.7599
mAP@0.5:0.95 (val)  : 0.5120
Précision moy (val) : 0.7756
Rappel moy (val)    : 0.6655


In [2]:
metrics = model.val(data=os.path.join(base_path, "data.yaml"), imgsz=640, split="val", conf=conf_threshold)


Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)


val: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:21<00:00,  5.34s/it]


                   all         63        336      0.776      0.666       0.76      0.512
                  Fito         33         45      0.804       0.64      0.744      0.532
               Monilia         22         26      0.724      0.654      0.743      0.519
                  Sana         54        265      0.799      0.703      0.793      0.485
Speed: 1.2ms preprocess, 179.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\val13


In [3]:
# Évaluation avec model.val au même seuil (YOLOv8s)
print("\n📊 Comparaison avec model.val :")
print(f"mAP@0.5 (val)       : {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95 (val)  : {metrics.box.map:.4f}")
print(f"Précision moy (val) : {metrics.box.p.mean():.4f}")
print(f"Rappel moy (val)    : {metrics.box.r.mean():.4f}")


📊 Comparaison avec model.val :
mAP@0.5 (val)       : 0.7599
mAP@0.5:0.95 (val)  : 0.5120
Précision moy (val) : 0.7756
Rappel moy (val)    : 0.6655


Métriques avec seuil de confiance = 0.5 (predict):
Précision (weighted) : 0.7997
Rappel (weighted)    : 0.6114
F1-Score (weighted)  : 0.6928


IoU=3
Métriques avec seuil de confiance = 0.4 (predict):
Précision (weighted) : 0.6726
Rappel (weighted)    : 0.6266
F1-Score (weighted)  : 0.6488

Métriques avec seuil de confiance = 0.6 (predict):
Précision (weighted) : 0.8693
Rappel (weighted)    : 0.5508
F1-Score (weighted)  : 0.6743
Métriques avec seuil de confiance = 0.7 (predict):
Précision (weighted) : 0.9392
Rappel (weighted)    : 0.3801
F1-Score (weighted)  : 0.5405



 Métriques avec seuil de confiance = 0.5 (predict)train2:
Précision (weighted) : 0.5902
Rappel (weighted)    : 0.5782
F1-Score (weighted)  : 0.5834

Métriques avec seuil de confiance = 0.5 (predict)train6:
Précision (weighted) : 0.7552
Rappel (weighted)    : 0.5771
F1-Score (weighted)  : 0.6541

Métriques avec seuil de confiance = 0.4 (predict) train6:
Précision (weighted) : 0.6258
Rappel (weighted)    : 0.5829
F1-Score (weighted)  : 0.6036

In [9]:
import os
import glob
import numpy as np
import pandas as pd
from ultralytics import YOLO
import torch

# Paramètres
base_path = "detection"
train_images_path = os.path.join(base_path, "images/val")
model_path = "runs/detect/train6/weights/best.pt"  # Chemin vers YOLOv8s
conf_threshold = 0.6  # Seuil de confiance optimal (basé sur F1-score)
iou_threshold = 0.45  # Seuil IoU pour NMS
iou_match_threshold = 0.3  # Seuil IoU pour appariement des boîtes
output_csv = "yolo_detections.csv"  # Fichier de sortie

# Charger le modèle YOLOv8s
model = YOLO(model_path)

# Récupérer les images d'entraînement
train_images = glob.glob(os.path.join(train_images_path, "*.jpg"))

# Fonction pour lire les annotations (vérités terrain)
def read_yolo_labels(label_path):
    labels = []
    bboxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                labels.append(class_id)
                bboxes.append([x_center, y_center, width, height])
    return labels, bboxes

# Fonction pour calculer l'IoU entre deux boîtes (format YOLO: [x_center, y_center, width, height])
def calculate_iou(box1, box2):
    # Convertir en [x_min, y_min, x_max, y_max]
    x1_min = box1[0] - box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_max = box1[1] + box1[3] / 2
    
    x2_min = box2[0] - box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_max = box2[1] + box2[3] / 2
    
    # Calculer l'intersection
    x_inter_min = max(x1_min, x2_min)
    y_inter_min = max(y1_min, y2_min)
    x_inter_max = min(x1_max, x2_max)
    y_inter_max = min(y1_max, y2_max)
    
    inter_area = max(0, x_inter_max - x_inter_min) * max(0, y_inter_max - y_inter_min)
    
    # Calculer l'union
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = area1 + area2 - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

# Liste pour stocker les résultats
results = []

# Effectuer les prédictions sur les images d'entraînement
for img_path in train_images:
    # Prédire avec YOLO
    results_yolo = model.predict(img_path, conf=conf_threshold, iou=iou_threshold, imgsz=640, verbose=False)
    
    # Récupérer les prédictions
    pred_classes = []
    pred_bboxes = []
    for result in results_yolo:
        if result.boxes is not None:
            pred_classes.extend(result.boxes.cls.cpu().numpy().astype(int))
            pred_bboxes.extend(result.boxes.xywhn.cpu().numpy())  # xywh normalisé
    
    # Récupérer les vérités terrain
    label_path = img_path.replace("images/val", "labels/val").replace(".jpg", ".txt")
    gt_classes, gt_bboxes = read_yolo_labels(label_path)
    
    # Appariement des boîtes basé sur l'IoU
    used_preds = set()
    used_gts = set()
    
    if pred_bboxes and gt_bboxes:
        # Calculer la matrice IoU
        iou_matrix = np.zeros((len(pred_bboxes), len(gt_bboxes)))
        for i, pred_box in enumerate(pred_bboxes):
            for j, gt_box in enumerate(gt_bboxes):
                iou_matrix[i, j] = calculate_iou(pred_box, gt_box)
        
        # Appariement glouton : associer les boîtes avec l'IoU le plus élevé
        while np.max(iou_matrix) >= iou_match_threshold:
            max_idx = np.unravel_index(np.argmax(iou_matrix), iou_matrix.shape)
            pred_idx, gt_idx = max_idx
            
            # Ajouter le résultat si les classes sont valides
            pred_class = pred_classes[pred_idx]
            gt_class = gt_classes[gt_idx]
            pred_box = pred_bboxes[pred_idx]
            
            # Stocker les informations
            results.append({
                "image_path": img_path,
                "pred_x_center": pred_box[0],
                "pred_y_center": pred_box[1],
                "pred_width": pred_box[2],
                "pred_height": pred_box[3],
                "pred_class": pred_class,
                "gt_class": gt_class,
                "iou": iou_matrix[max_idx]
            })
            
            used_preds.add(pred_idx)
            used_gts.add(gt_idx)
            iou_matrix[pred_idx, :] = -1
            iou_matrix[:, gt_idx] = -1
    
    # Ajouter les prédictions non appariées (optionnel, si vous voulez les inclure)
    for i, pred_class in enumerate(pred_classes):
        if i not in used_preds:
            pred_box = pred_bboxes[i]
            results.append({
                "image_path": img_path,
                "pred_x_center": pred_box[0],
                "pred_y_center": pred_box[1],
                "pred_width": pred_box[2],
                "pred_height": pred_box[3],
                "pred_class": pred_class,
                "gt_class": None,  # Pas de correspondance
                "iou": 0.0
            })
    
    # Ajouter les vérités terrain non appariées (optionnel)
    for i, gt_class in enumerate(gt_classes):
        if i not in used_gts:
            gt_box = gt_bboxes[i]
            results.append({
                "image_path": img_path,
                "pred_x_center": None,
                "pred_y_center": None,
                "pred_width": None,
                "pred_height": None,
                "pred_class": None,  # Pas de prédiction
                "gt_class": gt_class,
                "iou": 0.0
            })

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Sauvegarder les résultats dans un fichier CSV
results_df.to_csv(output_csv, index=False)
print(f"✅ Résultats sauvegardés dans {output_csv}")

# Afficher un aperçu des résultats
print("\n📊 Aperçu des résultats :")
print(results_df.head())

# Débogage : Statistiques sur les appariements
total_predictions = len(results_df[results_df["pred_class"].notnull()])
total_gt = len(results_df[results_df["gt_class"].notnull()])
matched = len(results_df[(results_df["pred_class"].notnull()) & (results_df["gt_class"].notnull())])
print(f"\n📊 Statistiques :")
print(f"Total prédictions : {total_predictions}")
print(f"Total vérités terrain : {total_gt}")
print(f"Appariements réussis : {matched}")
print(f"Pourcentage d'appariement : {matched / total_gt * 100:.2f}%")

✅ Résultats sauvegardés dans yolo_detections.csv

📊 Aperçu des résultats :
                         image_path  pred_x_center  pred_y_center  pred_width  \
0  detection\images/val\Fito104.jpg       0.324250       0.603014    0.081497   
1  detection\images/val\Fito104.jpg       0.212678       0.535426    0.092308   
2  detection\images/val\Fito104.jpg       0.626691       0.714324    0.190470   
3  detection\images/val\Fito104.jpg            NaN            NaN         NaN   
4  detection\images/val\Fito104.jpg            NaN            NaN         NaN   

   pred_height  pred_class  gt_class       iou  
0     0.171268         2.0       2.0  0.858043  
1     0.112222         2.0       2.0  0.812791  
2     0.348745         2.0       2.0  0.731813  
3          NaN         NaN       0.0  0.000000  
4          NaN         NaN       2.0  0.000000  

📊 Statistiques :
Total prédictions : 213
Total vérités terrain : 336
Appariements réussis : 206
Pourcentage d'appariement : 61.31%


In [ ]:

# --------------------------
# 3. Entraînement du modèle YOLOv8
# --------------------------
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

models = YOLO("yolov8s.pt")  # 🔁 change "s" par "n", "m", ou "l" selon ta capacité

models.train(
    data=os.path.join(base_path, "data.yaml"),  # Dataset
    epochs=50,         # 🔥 Plus d'époques pour bien converger
    imgsz=768,          # 🔥 Résolution plus grande pour mieux voir les cabosses
    batch=8,
    degrees=10,         # ➔ Petites rotations
    translate=0.1,      # ➔ Petits décalages
    scale=0.7,          # ➔ Zoom aléatoire
    shear=2.0,          # ➔ Cisaillements légers
    perspective=0.001,  # ➔ Perspective (effet 3D)
    flipud=0.1,         # ➔ Petit flip vertical
    fliplr=0.5,         # ➔ Flip horizontal
    hsv_h=0.015,        # ➔ Variation de teinte
    hsv_s=0.7,          # ➔ Variation de saturation (important pour feuillage vs cabosse)
    hsv_v=0.4,          # ➔ Variation de luminosité
    mosaic=1.0,         # ➔ Collage d'images pour simuler scènes complexes
    mixup=0.2,          # ➔ Mélange partiel d'images
    workers=2,          # ➔ Nombre de processus de chargement
    patience=20,        # ➔ Early stopping si pas de progrès
    verbose=True        # ➔ Afficher plus de détails pendant l'entraînement
)

New https://pypi.org/project/ultralytics/8.3.117 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=detection\data.yaml, epochs=50, time=None, patience=20, batch=8, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

train: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\train.cache... 249 images, 0 backgrounds, 0 corrupt: 100%|██████████| 249/249 [00:00<?, ?it/s]
val: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]

Plotting labels to runs\detect\train3\labels.jpg... 



optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 768 train, 768 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/32 [00:00<?, ?it/s]

In [7]:
# --------------------------
# 4. Évaluation du modèle
# --------------------------
best_model_path = "runs/detect/train6/weights/best.pt"  

models= YOLO(best_model_path)

metricss = models.val(data=os.path.join(base_path, "data.yaml"), imgsz=768, split="val")

Ultralytics 8.3.89  Python-3.12.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\labels\val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:27<00:00,  6.99s/it]


                   all         63        336      0.781       0.63      0.744      0.456
                  Fito         33         45      0.856      0.556      0.684      0.439
               Monilia         22         26      0.657      0.692      0.746      0.499
                  Sana         54        265       0.83      0.643      0.803      0.428
Speed: 4.1ms preprocess, 278.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val3


In [8]:


# --------------------------
# 5. Affichage des métriques
# --------------------------
print("\n📊 Évaluation du modèle :")
print(f"mAP@0.5       : {metricss.box.map50:.4f}")
print(f"mAP@0.5:0.95  : {metricss.box.map:.4f}")
print(f"Précision moy : {metricss.box.p.mean():.4f}")
print(f"Recall moy    : {metricss.box.r.mean():.4f}")



📊 Évaluation du modèle :
mAP@0.5       : 0.7445
mAP@0.5:0.95  : 0.4557
Précision moy : 0.7808
Recall moy    : 0.6303


In [18]:
import os
import numpy as np
from ultralytics import YOLO
from mean_average_precision import MetricBuilder

# --------------------------
# 1. Chargement du modèle
# --------------------------
base_path = "detection"
best_model_path = "runs/detect/train6/weights/best.pt"

model = YOLO(best_model_path)

# --------------------------
# 2. Prédictions avec seuil
# --------------------------
predictions = model.predict(
    source=os.path.join(base_path, "images/val"),
    conf=0.5,        # ➔ seuil de confiance appliqué
    iou=0.5,
    imgsz=768,
    save=False
)

# --------------------------
# 3. Initialisation de MetricBuilder
# --------------------------
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=3)

# --------------------------
# 4. Boucle d'évaluation
# --------------------------
for prediction in predictions:
    # Ground truth : charger les labels
    label_path = prediction.path.replace("images/val", "labels/val").replace(".jpg", ".txt")
    
    if not os.path.exists(label_path):
        continue
    
    labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls_id, x_center, y_center, width, height = map(float, parts)
            labels.append([cls_id, x_center, y_center, width, height])

    h, w = prediction.orig_shape
    targets = []
    for lbl in labels:
        cls_id, x_c, y_c, bw, bh = lbl
        x_min = (x_c - bw/2) * w
        y_min = (y_c - bh/2) * h
        x_max = (x_c + bw/2) * w
        y_max = (y_c + bh/2) * h
        targets.append([cls_id, x_min, y_min, x_max, y_max])

    # Prédictions
    preds = []
    if prediction.boxes.shape[0] > 0:
        for box in prediction.boxes.data.cpu().numpy():
            x_min, y_min, x_max, y_max, score, cls_id = box
            preds.append([cls_id, score, x_min, y_min, x_max, y_max])

    preds = np.array(preds)
    targets = np.array(targets)
    
    # Ajouter à MetricBuilder
    metric_fn.add(preds=preds, gts=targets)

# --------------------------
# 5. Résultats
# --------------------------
eval_results = metric_fn.value(iou_thresholds=np.array([0.5]))

# Vérification des clés disponibles
print("\n✅ Clés disponibles dans eval_results :", eval_results.keys())





image 1/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito104.jpg: 768x576 1 Fito, 5 Sanas, 311.4ms
image 2/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito12.jpg: 768x576 1 Monilia, 3 Sanas, 334.5ms
image 3/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito14.jpg: 768x576 1 Fito, 1 Sana, 310.7ms
image 4/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito19.jpg: 768x576 1 Sana, 290.7ms
image 5/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito2.jpg: 768x576 1 Fito, 2 Sanas, 327.4ms
image 6/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito23.jpg: 768x576 1 Fito, 6 Sanas, 340.0ms
image 7/63 c:\Users\AMED BAH\Desktop\COURS M2\Deep Learning\Projet_deepLearning\detection\images\val\Fito26.jpg: 768x576 2 Fitos, 

In [20]:
# --------------------------
# 5. Résultats corrigés
# --------------------------
map50 = eval_results[0.5]    # mAP@0.5
map_50_95 = eval_results['mAP']  # mAP@0.5:0.95 (moyenne globale)



# --------------------------
# 6. Affichage final
# --------------------------
print("\n📊 Évaluation manuelle après prédiction (conf=0.5) :")
print(f"mAP@0.5       : {map50:.4f}")
print(f"mAP@0.5:0.95  : {map_50_95:.4f}")




📊 Évaluation manuelle après prédiction (conf=0.5) :


TypeError: unsupported format string passed to dict.__format__

In [21]:
print(eval_results[0.5])
print(eval_results['mAP'])


{0: {'ap': 0.0, 'precision': array([], dtype=float64), 'recall': array([], dtype=float64)}, 1: {'ap': 0.0, 'precision': array([], dtype=float64), 'recall': array([], dtype=float64)}, 2: {'ap': 0.0, 'precision': array([], dtype=float64), 'recall': array([], dtype=float64)}}
0.0
